# Project 2: Topic Classification

In this project, you'll work with text data from newsgroup postings on a variety of topics. You'll train classifiers to distinguish between the topics based on the text of the posts. Whereas with digit classification, the input is relatively dense: a 28x28 matrix of pixels, many of which are non-zero, here we'll represent each document with a "bag-of-words" model. As you'll see, this makes the feature representation quite sparse -- only a few words of the total vocabulary are active in any given document. The bag-of-words assumption here is that the label depends only on the words; their order is not important.

The SK-learn documentation on feature extraction will prove useful:
http://scikit-learn.org/stable/modules/feature_extraction.html

Each problem can be addressed succinctly with the included packages -- please don't add any more. Grading will be based on writing clean, commented code, along with a few short answers.

As always, you're welcome to work on the project in groups and discuss ideas on the course wall, but please prepare your own write-up and write your own code.

In [15]:
# This tells matplotlib not to try opening a new window for each plot.
%matplotlib inline

# General libraries.
import re
import numpy as np
import matplotlib.pyplot as plt

# SK-learn libraries for learning.
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB

# SK-learn libraries for evaluation.
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import classification_report

# SK-learn library for importing the newsgroup data.
from sklearn.datasets import fetch_20newsgroups

# SK-learn libraries for feature extraction from text.
from sklearn.feature_extraction.text import *

import nltk

Load the data, stripping out metadata so that we learn classifiers that only use textual features. By default, newsgroups data is split into train and test sets. We further split the test so we have a dev set. Note that we specify 4 categories to use for this project. If you remove the categories argument from the fetch function, you'll get all 20 categories.

In [2]:
categories = ['alt.atheism', 'talk.religion.misc', 'comp.graphics', 'sci.space']
newsgroups_train = fetch_20newsgroups(subset='train',
                                      remove=('headers', 'footers', 'quotes'),
                                      categories=categories)
newsgroups_test = fetch_20newsgroups(subset='test',
                                     remove=('headers', 'footers', 'quotes'),
                                     categories=categories)

num_test = int(len(newsgroups_test.target) / 2)
test_data, test_labels = newsgroups_test.data[num_test:], newsgroups_test.target[num_test:]
dev_data, dev_labels = newsgroups_test.data[:num_test], newsgroups_test.target[:num_test]
train_data, train_labels = newsgroups_train.data, newsgroups_train.target

print('training label shape:', train_labels.shape)
print('test label shape:', test_labels.shape)
print('dev label shape:', dev_labels.shape)
print('labels names:', newsgroups_train.target_names)

training label shape: (2034,)
test label shape: (677,)
dev label shape: (676,)
labels names: ['alt.atheism', 'comp.graphics', 'sci.space', 'talk.religion.misc']


### Part 1:

For each of the first 5 training examples, print the text of the message along with the label.

In [3]:
#def P1(num_examples=5):
### STUDENT START ###
for x in range(5):
    print("Label:", train_labels[x], "-", newsgroups_train.target_names[int(train_labels[x])])
    print("Message:", train_data[x])
    print("")
### STUDENT END ###
#P1()

Label: 1 - comp.graphics
Message: Hi,

I've noticed that if you only save a model (with all your mapping planes
positioned carefully) to a .3DS file that when you reload it after restarting
3DS, they are given a default position and orientation.  But if you save
to a .PRJ file their positions/orientation are preserved.  Does anyone
know why this information is not stored in the .3DS file?  Nothing is
explicitly said in the manual about saving texture rules in the .PRJ file. 
I'd like to be able to read the texture rule information, does anyone have 
the format for the .PRJ file?

Is the .CEL file format available from somewhere?

Rych

Label: 3 - talk.religion.misc
Message: 

Seems to be, barring evidence to the contrary, that Koresh was simply
another deranged fanatic who thought it neccessary to take a whole bunch of
folks with him, children and all, to satisfy his delusional mania. Jim
Jones, circa 1993.


Nope - fruitcakes like Koresh have been demonstrating such evil corruption
fo

### Part 2:

Use CountVectorizer to turn the raw training text into feature vectors. You should use the fit_transform function, which makes 2 passes through the data: first it computes the vocabulary ("fit"), second it converts the raw text into feature vectors using the vocabulary ("transform").

The vectorizer has a lot of options. To get familiar with some of them, write code to answer these questions:

a. The output of the transform (also of fit_transform) is a sparse matrix: http://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.sparse.csr_matrix.html. What is the size of the vocabulary? What is the average number of non-zero features per example? What fraction of the entries in the matrix are non-zero? Hint: use "nnz" and "shape" attributes.

b. What are the 0th and last feature strings (in alphabetical order)? Hint: use the vectorizer's get_feature_names function.

c. Specify your own vocabulary with 4 words: ["atheism", "graphics", "space", "religion"]. Confirm the training vectors are appropriately shaped. Now what's the average number of non-zero features per example?

d. Instead of extracting unigram word features, use "analyzer" and "ngram_range" to extract bigram and trigram character features. What size vocabulary does this yield?

e. Use the "min_df" argument to prune words that appear in fewer than 10 documents. What size vocabulary does this yield?

f. Using the standard CountVectorizer, what fraction of the words in the dev data are missing from the vocabulary? Hint: build a vocabulary for both train and dev and look at the size of the difference.

In [4]:
#def P2():
### STUDENT START ###
# Turns the training data into feature vectors, default parameters
CV = CountVectorizer()
train_fit = CV.fit_transform(train_data)
# Exmaining the default vocabulary formed by countvectorizer - a
print("The size of the training data vocabulary is:", train_fit.shape[1])
print("Average number of non-zero per example:", round(train_fit.nnz / len(train_data), 3))
print("Percentage of non-zeros to total is:", round((train_fit.nnz / np.size(train_fit.toarray())) * 100, 3))
print("")
# Prints first and last feature string - b
print("First and last feature strings:", [CV.get_feature_names()[i] for i in (0, -1)])
print("")
# Adding customer vocabulary - c
my_vocab = ["atheism", "graphics", "space", "religion"]
vocab_CV = CountVectorizer(vocabulary=my_vocab)
vocab_fit = vocab_CV.fit_transform(train_data)
print("Custom vocabulary matrix shape:", vocab_fit.shape)
print("Average number of non-zeros per example, defined vocabulary:", round(vocab_fit.nnz / len(train_data), 3))
print("")
# Testing bigram and trigram features - d
bigram_CV = CountVectorizer(analyzer="word", ngram_range=(2,2))
bigram_fit = bigram_CV.fit_transform(train_data)
print("Bigram vocabulary size:", bigram_fit.shape[1])
trigram_CV = CountVectorizer(analyzer="word", ngram_range=(3,3))
trigram_fit = trigram_CV.fit_transform(train_data)
print("Trigram vocabulary size:", trigram_fit.shape[1])
print("")
# Restricting types of words added to vocabulary based on frequency - e
min_CV = CountVectorizer(min_df = 10)
min_fit = min_CV.fit_transform(train_data)
print("Vocabulary size using 'min_df':", min_fit.shape[1])
print("")
# Comparing train_data with dev_data - f
dev_fit = CV.fit_transform(dev_data)
print("Percentage vocabulary missing in dev vocabulary:", 
      round(100 * (train_fit.shape[1] - dev_fit.shape[1]) / train_fit.shape[1], 3))
### STUDENT END ###
#P2()

The size of the training data vocabulary is: 26879
Average number of non-zero per example: 96.706
Percentage of non-zeros to total is: 0.36

First and last feature strings: ['00', 'zyxel']

Average number of non-zeros per example, defined vocabulary: 0.268

Bigram vocabulary size: 194891
Trigram vocabulary size: 315692

Vocabulary size using 'min_df': 3064

Percentage vocabulary missing in dev vocabulary: 39.559


### Part 3:

Use the default CountVectorizer options and report the f1 score (use metrics.f1_score with average="weighted") for a k nearest neighbors classifier; find the optimal value for k. Also fit a Multinomial Naive Bayes model and find the optimal value for alpha. Finally, fit a logistic regression model and find the optimal value for the regularization strength C using l2 regularization. A few questions:

* Why doesn't nearest neighbors work well for this problem?
* Any ideas why logistic regression doesn't work as well as Naive Bayes?
* Logistic regression estimates a weight vector for each class, which you can access with the coef\_ attribute. Output the sum of the squared weight values for each class for each setting of the C parameter. Briefly explain the relationship between the sum and the value of C.

In [10]:
#def P3():
### STUDENT START ###
# Setting default parameters for CountVectorizer
CountVect = CountVectorizer()
CountVect_train = CountVect.fit_transform(train_data)
CV_dev = CountVect.transform(dev_data)

# # k-Nearest Neighbors model
kNN = KNeighborsClassifier(n_neighbors=91)
kNN.fit(CountVect_train, train_labels)
kNN_predict = kNN.predict(CV_dev)
kNN_f1 = metrics.f1_score(dev_labels, kNN_predict, average='weighted')
print("The f1 score for k-Nearest Neighbors is:", round(kNN_f1, 4))

# # Multinomial Naive Bayes model
MNNB = MultinomialNB(alpha=0.1)
MNNB.fit(CountVect_train, train_labels)
MNNB_pred = MNNB.predict(CV_dev)
MNNB_f1 = metrics.f1_score(dev_labels, MNNB_pred, average='weighted')
print("The f1 score for Multinominal Naive Bayes is:", round(MNNB_f1, 4))

# # Logistic Regression Model - 'l2' regularization
LogReg = LogisticRegression(C=0.1, max_iter=400, penalty = 'l2')   # max_iter is set due to getting error on default
LogReg.fit(CountVect_train, train_labels)
LogReg_pred = LogReg.predict(CV_dev)
LogReg_f1 = metrics.f1_score(dev_labels, LogReg_pred, average='weighted')
print("The f1 score for Logistic Regression is:", round(LogReg_f1, 4))
print("")
    
print("Sum of squared weights for each class:")
print(LogReg.coef_)
### STUDENT END ###
#P3()

The f1 score for k-Nearest Neighbors is: 0.4535
The f1 score for Multinominal Naive Bayes is: 0.7903
The f1 score for Logistic Regression is: 0.6971
Sum of squared weights for each class:
[[-3.10351069e-02  2.36524719e-02 -2.38195366e-05 ... -1.39132271e-04
  -2.78264541e-04 -1.30582969e-04]
 [ 6.85519276e-02  2.66661125e-02 -2.30175293e-03 ... -4.48728658e-04
  -8.97457317e-04  4.51644208e-03]
 [-1.13419621e-02 -5.18815746e-02  2.91071004e-03 ...  6.87273777e-04
   1.37454755e-03 -4.29955775e-03]
 [-2.61748585e-02  1.56299031e-03 -5.85137568e-04 ... -9.94128474e-05
  -1.98825695e-04 -8.63013594e-05]]


ANSWER:

### Part 4:

Train a logistic regression model. Find the 5 features with the largest weights for each label -- 20 features in total. Create a table with 20 rows and 4 columns that shows the weight for each of these features for each of the labels. Create the table again with bigram features. Any surprising features in this table?

In [5]:
#def P4():
### STUDENT START ###
# Transforming the raw data into tokens - default parameters
CV = CountVectorizer()
CV_train = CV.fit_transform(train_data)
# Bigram parameters for CountVectorizer
bigram_CV = CountVectorizer(analyzer='word', ngram_range=(2,2))
bigram_train = bigram_CV.fit_transform(train_data)

# Logistic Regression model - default CV
LR = LogisticRegression(C=0.1, max_iter=400)
LR.fit(CV_train, train_labels)
# Finding largest weights for each feature
max_post = []
for i in range(LR.coef_.shape[0]):
    arr_abs = np.absolute(LR.coef_[i])
    temp_list = np.argpartition(-arr_abs, 5)[:5]
    temp_values = [LR.coef_[i][n] for n in list(temp_list)]
    max_post.append(temp_values)

# Table for default Logistic Regression

# Bigram - Logistic Regression model
bigram_LR = LogisticRegression(C=0.1, max_iter=400)
bigram_LR.fit(bigram_train, train_labels)
# Weights for each feature bigram model
max_bigram = []
for i in range(bigram_LR.coef_.shape[0]):
    arr_abs = np.absolute(bigram_LR.coef_[i])
    temp_list = np.argpartition(-arr_abs, 5)[:5]
    temp_values = [bigram_LR.coef_[i][n] for n in list(temp_list)]
    max_bigram.append(temp_values)

print(max_post)
print(max_bigram)
### STUDENT END ###
#P4()

[[-0.35746130312790586, 0.3787161577035587, 0.4069874652628062, 0.3526107828527006, 0.33875261544785934], [0.5109066933708672, 0.7665558965309961, 0.5000166797390253, 0.42052873632827903, 0.3935201363547246], [0.9981901187052326, 0.45292361382686946, 0.4399952048300425, -0.36616656274099757, 0.35211951245946216], [-0.3465440710038419, 0.41476308466504297, 0.4211996303967779, 0.4094094685074654, 0.32771638161239586]]
[[0.267651421620217, 0.24782393421861987, 0.23831936623670438, 0.2355559628600719, -0.23223818602450222], [0.5515306941119664, 0.42513393357741613, 0.36442352316024224, 0.3461502750683927, 0.31111885219752317], [0.39264684997125243, 0.2845869215789034, 0.4018298108712746, 0.28097394289091776, 0.27721470417765326], [0.2065551389409348, 0.24638298955825322, 0.2157835445753937, 0.21213511181312691, 0.28700098832730525]]


ANSWER:

### Part 5:

Try to improve the logistic regression classifier by passing a custom preprocessor to CountVectorizer. The preprocessing function runs on the raw text, before it is split into words by the tokenizer. Your preprocessor should try to normalize the input in various ways to improve generalization. For example, try lowercasing everything, replacing sequences of numbers with a single token, removing various other non-letter characters, and shortening long words. If you're not already familiar with regular expressions for manipulating strings, see https://docs.python.org/2/library/re.html, and re.sub() in particular. With your new preprocessor, how much did you reduce the size of the dictionary?

For reference, I was able to improve dev F1 by 2 points.

In [58]:
#def empty_preprocessor(s):
#    return s

#def better_preprocessor(s):
### STUDENT START ###
# Default CountVectorizer to compare the change
CV = CountVectorizer()
CV_train = CV.fit_transform(train_data)
CV_dev = CV.transform(dev_data)
print("The size of the training data vocabulary is:", CV_train.shape[1])

LR = LogisticRegression(C=0.1, max_iter=400, penalty = 'l2')
LR.fit(CV_train, train_labels)
LR_pred = LR.predict(CV_dev)

LR_f1 = metrics.f1_score(dev_labels, LR_pred, average='weighted')
print("The f1 score for Logistic Regression is:", round(LR_f1, 4))
### STUDENT END ###
print("")
#def P5():
### STUDENT START ###
# Function for preprocessing the data
porter_stemmer = nltk.stem.PorterStemmer()
def text_cleaning(text):
    # Lowercase every word
    text = text.lower()
    # Noise Reducing - Cleaning up words
    text = re.sub("\\W|\\d", " ", text)   # Removing digits and non-ascii characters
    # White space removeal
    text=text.strip()
    # Stop word removal
    stop_words = set(nltk.corpus.stopwords.words('english'))
    words = re.split("\\s+",text)
    filt_text = [w for w in words if not w in stop_words] 
    # Stemming words
    stem_words = [porter_stemmer.stem(word=word) for word in filt_text]
    stem_text = " ".join(stem_words)
    
    return stem_text
                      
# Preprocessed training data
pre_CV = CountVectorizer(preprocessor=text_cleaning)
pre_train = pre_CV.fit_transform(train_data)
pre_dev = pre_CV.transform(dev_data)
print("The size of the preprocessed training data vocabulary is:", pre_train.shape[1])

pre_LR = LogisticRegression(C=0.1, max_iter=400, penalty='l2')
pre_LR.fit(pre_train, train_labels)
pre_LR_predict = pre_LR.predict(pre_dev)

pre_LR_f1 = metrics.f1_score(dev_labels, pre_LR_predict, average='weighted')
print("The f1 score for the preprocessed Logistic Regression is:", round(pre_LR_f1, 4))
### STUDENT END ###
#P5()


The size of the preprocessed training data vocabulary is: 17137
The f1 score for the preprocessed Logistic Regression is: 0.7136


### Part 6:

The idea of regularization is to avoid learning very large weights (which are likely to fit the training data, but not generalize well) by adding a penalty to the total size of the learned weights. That is, logistic regression seeks the set of weights that minimizes errors in the training data AND has a small size. The default regularization, L2, computes this size as the sum of the squared weights (see P3, above). L1 regularization computes this size as the sum of the absolute values of the weights. The result is that whereas L2 regularization makes all the weights relatively small, L1 regularization drives lots of the weights to 0, effectively removing unimportant features.

Train a logistic regression model using a "l1" penalty. Output the number of learned weights that are not equal to zero. How does this compare to the number of non-zero weights you get with "l2"? Now, reduce the size of the vocabulary by keeping only those features that have at least one non-zero weight and retrain a model using "l2".

Make a plot showing accuracy of the re-trained model vs. the vocabulary size you get when pruning unused features by adjusting the C parameter.

Note: The gradient descent code that trains the logistic regression model sometimes has trouble converging with extreme settings of the C parameter. Relax the convergence criteria by setting tol=.015 (the default is .0001).

In [109]:
#def P6():
    # Keep this random seed here to make comparison easier.
np.random.seed(0)

    ### STUDENT START ###
CV = CountVectorizer()
train_CV = CV.fit_transform(train_data)
dev_CV = CV.transform(dev_data)
    
l1_LR = LogisticRegression(C=0.1, max_iter=500, penalty='l1', solver='liblinear', tol = 0.015)
l1_LR.fit(train_CV, train_labels)
l1_LR_predict = l1_LR.predict(dev_CV)

non_zero_matrix = l1_LR.coef_ != 0    # Checks which feature contains zero weights
nz_postition = non_zero_matrix.any(axis=0)    # Takes the features that has non_zero weights
print("Number of non-zero weights for l1 Logistic Regression:", 
      sum(nz_postition))
# Selecting the features that are only non-zero
train_trim = train_CV[:, nz_postition]
dev_trim = dev_CV[:, nz_postition]

l2_LR = LogisticRegression(C=0.1, max_iter=500, penalty="l2", tol=0.015)
l2_LR.fit(train_trim, train_labels)
l2_LR_predict = l2_LR.predict(dev_trim)

print("F1-score of 'l1' Logistic Regression model:",
      round(metrics.f1_score(dev_labels, l1_LR_predict, average='weighted'), 3))
print("F1-score of 'l2' Logistic Regression model:",
      round(metrics.f1_score(dev_labels, l2_LR_predict, average='weighted'), 3))
    ### STUDENT END ###
#P6()

Number of non-zero weights for l1 Logistic Regression: 218
F1-score of 'l1' Logistic Regression model: 0.659
F1-score of 'l2' Logistic Regression model: 0.683


### Part 7:

Use the TfidfVectorizer -- how is this different from the CountVectorizer? Train a logistic regression model with C=100.

Make predictions on the dev data and show the top 3 documents where the ratio R is largest, where R is:

maximum predicted probability / predicted probability of the correct label

What kinds of mistakes is the model making? Suggest a way to address one particular issue that you see.

In [94]:
#def P7():
### STUDENT START ###
# Setting the TfidVectorizer with default settings
TV = TfidfVectorizer()
train_TV = TV.fit_transform(train_data)
dev_TV = TV.transform(dev_data)

TV_LR = LogisticRegression(C=100, max_iter=500)
TV_LR.fit(train_TV, train_labels)
TV_predict = TV_LR.predict(dev_TV)
print("f1 score using TfidfVectorizer:", 
      round(metrics.f1_score(dev_labels, TV_predict, average='weighted'), 4))
# Obtaining the max predicted probabilities
prob_dev = TV_LR.predict_proba(dev_TV)
max_prob = prob_dev.max(axis=1)


## STUDENT END ###
#P7()

f1 score using TfidfVectorizer: 0.768


ANSWER:

### Part 8 EXTRA CREDIT:

Try implementing one of your ideas based on your error analysis. Use logistic regression as your underlying model.